In [2]:
# enc.decode(out[i])  is where the decoding of the differnet integers happens. 
import pandas as pd

import json
with open('encoder.json', 'r') as f:
    data = json.load(f)

    def bytes_to_unicode():
        """
        Returns list of utf-8 byte and a corresponding list of unicode strings.
        The reversible bpe codes work on unicode strings.
        This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
        When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
        This is a signficant percentage of your normal, say, 32K bpe vocab.
        To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
        And avoids mapping to whitespace/control characters the bpe code barfs on.
        """
        bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
        cs = bs[:]
        n = 0
        for b in range(2**8):
            if b not in bs:
                bs.append(b)
                cs.append(2**8+n)
                n += 1
        cs = [chr(n) for n in cs]
        return dict(zip(bs, cs))

IndentationError: expected an indented block (<ipython-input-2-b0ced0b1968d>, line 17)

In [36]:
decoder = {v:k for k,v in data.items()}

In [51]:
byte_encoder = bytes_to_unicode()
byte_decoder = {v:k for k, v in byte_encoder.items()}


{'!': 33,
 '"': 34,
 '#': 35,
 '$': 36,
 '%': 37,
 '&': 38,
 "'": 39,
 '(': 40,
 ')': 41,
 '*': 42,
 '+': 43,
 ',': 44,
 '-': 45,
 '.': 46,
 '/': 47,
 '0': 48,
 '1': 49,
 '2': 50,
 '3': 51,
 '4': 52,
 '5': 53,
 '6': 54,
 '7': 55,
 '8': 56,
 '9': 57,
 ':': 58,
 ';': 59,
 '<': 60,
 '=': 61,
 '>': 62,
 '?': 63,
 '@': 64,
 'A': 65,
 'B': 66,
 'C': 67,
 'D': 68,
 'E': 69,
 'F': 70,
 'G': 71,
 'H': 72,
 'I': 73,
 'J': 74,
 'K': 75,
 'L': 76,
 'M': 77,
 'N': 78,
 'O': 79,
 'P': 80,
 'Q': 81,
 'R': 82,
 'S': 83,
 'T': 84,
 'U': 85,
 'V': 86,
 'W': 87,
 'X': 88,
 'Y': 89,
 'Z': 90,
 '[': 91,
 '\\': 92,
 ']': 93,
 '^': 94,
 '_': 95,
 '`': 96,
 'a': 97,
 'b': 98,
 'c': 99,
 'd': 100,
 'e': 101,
 'f': 102,
 'g': 103,
 'h': 104,
 'i': 105,
 'j': 106,
 'k': 107,
 'l': 108,
 'm': 109,
 'n': 110,
 'o': 111,
 'p': 112,
 'q': 113,
 'r': 114,
 's': 115,
 't': 116,
 'u': 117,
 'v': 118,
 'w': 119,
 'x': 120,
 'y': 121,
 'z': 122,
 '{': 123,
 '|': 124,
 '}': 125,
 '~': 126,
 '¡': 161,
 '¢': 162,
 '£': 163,

In [24]:
import pickle
import numpy as np 
import gzip
all_logits = pickle.load(gzip.open('gpt-2_output/all_logits5000_word_prompts.pickle.gz', 'rb'))
rand_selection = pickle.load(gzip.open('gpt-2_output/prompt_rand_selections_5000_word_prompts.pickle.gz', 'rb'))
df=pd.read_csv('test_dataframe_200primer.csv')

In [25]:
df.iloc[rand_selection] # checking that the random selection performed the way it was supposed to. 

,test_source,test_target,test_target_200,Prompt
5139,[WP] You look out at the bright lights of the ...,The lights dance across the evening sky. You c...,The lights dance across the evening sky. You c...,[WP] You look out at the bright lights of the ...
3768,[FF] - 250 Words; 2 months Reddit Gold prize,"Could this be it? I could have sworn, no... no...","Could this be it? I could have sworn, no... no...",[FF] - 250 Words; 2 months Reddit Gold prize \...
3912,[WP] The shortest story ever told. Once upon a...,"Once upon a time, there was a magical place wh...","Once upon a time, there was a magical place wh...",[WP] The shortest story ever told. Once upon a...
14879,"[WP] Stockholm syndrome, but the other way aro...",``Here it is. 2 million dollars in unmarked bi...,``Here it is. 2 million dollars in unmarked bi...,"[WP] Stockholm syndrome, but the other way aro..."
4848,[WP] Your grandparents always playfully bicker...,It wasn ’ t Swedish. Well I have no way of act...,It wasn ’ t Swedish. Well I have no way of act...,[WP] Your grandparents always playfully bicker...


In [56]:
ind = 0
tokens = []
for j in range( all_logits[0].shape[2]):
    tokens.append(np.argmax(all_logits[0][ind, :, j]))
errors='replace'
text = ''.join([decoder[token] for token in tokens])
text = bytearray([byte_decoder[c] for c in text]).decode('utf-8', errors=errors)
text   

' you ever feel like that? \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n'

In [58]:
'|'.join([decoder[token] for token in tokens])

'Ġyou|Ġever|Ġfeel|Ġlike|Ġthat|?|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ|Ġ|Ċ'

In [60]:
# If I use the detokenized version then I should be good. you know i can use the moses encoder on the predicted text. and then compare the originals? 

### Validation where ground truth is known: 

In [38]:
li = [ 286,  428, 1295,   11,  475,  356, 1183, 1011, 1337,  286,  340,   13,  921, 1183,
 1560,  262, 1644,  345, 1239, 2982,  286,  428, 1295,   11,  475,  356, 1183, 1011,
 1337,  286,  340,   13,  921, 1183, 1560,  262, 1644,  345, 1239, 2982,  286,  428,
 1295,   11,  475,  356, 1183, 1011, 1337,  286,  340,   13,  921, 1183, 1560,  262,
 1644,  345, 1239, 2982,  286,  428, 1295,   11,  475,  356, 1183, 1011, 1337,  286,
  340,   13,  921, 1183, 1560,  262, 1644,  345, 1239, 2982,  286,  428, 1295,   11,
  475,  356, 1183, 1011, 1337,  286,  340,   13,  921, 1183, 1560,  262, 1644,  345,
 1239, 2982]


li = np.asarray(li)

array([ 286,  428, 1295,   11,  475,  356, 1183, 1011, 1337,  286,  340,
         13,  921, 1183, 1560,  262, 1644,  345, 1239, 2982,  286,  428,
       1295,   11,  475,  356, 1183, 1011, 1337,  286,  340,   13,  921,
       1183, 1560,  262, 1644,  345, 1239, 2982,  286,  428, 1295,   11,
        475,  356, 1183, 1011, 1337,  286,  340,   13,  921, 1183, 1560,
        262, 1644,  345, 1239, 2982,  286,  428, 1295,   11,  475,  356,
       1183, 1011, 1337,  286,  340,   13,  921, 1183, 1560,  262, 1644,
        345, 1239, 2982,  286,  428, 1295,   11,  475,  356, 1183, 1011,
       1337,  286,  340,   13,  921, 1183, 1560,  262, 1644,  345, 1239,
       2982])

In [54]:
errors='replace'
text = ''.join([decoder[token] for token in li])
text = bytearray([byte_decoder[c] for c in text]).decode('utf-8', errors=errors)
text

" of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard"

Index output pre and post decoding: 



[ 286  428 1295   11  475  356 1183 1011 1337  286  340   13  921 1183
 1560  262 1644  345 1239 2982  286  428 1295   11  475  356 1183 1011
 1337  286  340   13  921 1183 1560  262 1644  345 1239 2982  286  428
 1295   11  475  356 1183 1011 1337  286  340   13  921 1183 1560  262
 1644  345 1239 2982  286  428 1295   11  475  356 1183 1011 1337  286
  340   13  921 1183 1560  262 1644  345 1239 2982  286  428 1295   11
  475  356 1183 1011 1337  286  340   13  921 1183 1560  262 1644  345
 1239 2982]
======================================== SAMPLE 1 ========================================
 of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard of this place, but we'll take care of it. You'll tell the police you never heard